In [1]:
import telebot 
from telebot import types
token='1058752933:AAGVFUetCtvTR3hePsSBclMbekIMAQDwwUs'
START, GIFT, SIZE, NAME, PHONE, CITY, FINISH =range(7)

In [2]:

from collections import defaultdict
USER_STATE=defaultdict(lambda: START)

def get_state(message):
    return USER_STATE[message.chat.id]
def update_state(message, state):
     USER_STATE[message.chat.id]=state
        
PRODUCTS=defaultdict(lambda: {})
def update_product(user_id, key, value):
    PRODUCTS[user_id][key]=value
def get_product(user_id):
    return PRODUCTS[user_id]
    

In [3]:
bot=telebot.TeleBot(token)
currencies=['euro', 'dollar']
#gifts={'Коллаж': 1, 'Электронный коллаж': 2, 'Dream Art': 3, 'Печать без обработки': 4}
gifts={1: 'Коллаж на холсте', 2: 'Портрет в стиле Dream Art на холсте', 3: 'Портрет в стиле Love is на холсте ', 4: 'Коллаж/портрет в электронном виде ',  5: 'Печать Вашего фото на холсте без обработки', 6: 'Нужен совет'}
#sizes={'30*30': 1, '30*40':2, '40*40':3, '40*50':4, '50*50':5, '50*60':6, '60*60':7, '40*60':8, '50*70':9}
sizes={1:'30*30', 2:'30*40', 3:'40*40', 4:'40*50', 5:'50*50', 6:'50*60', 7:'60*60', 8:'40*60', 9:'50*70', 10: 'Нужен совет'}
#cities={'Москва': 1, 'Другой город': 2}
cities={1:'Москва', 2:'Другой город'}
yes_no={1:'Отправить заявку', 2: 'Исправить информацию'}
who={1:'Маме', 2: 'Папа', 3:'Девушке', 4:'Парню', 5:'Другу', 6:'Секрет', 7:'Другое'}
start=1


@bot.message_handler(commands=['start', 'help'])
def beginning(message):
    keyboard=create_keyboard_gifts()
    bot.send_message(message.chat.id, text="Выберите свой первый подарок. \nС нашей коллекцией Вы можете ознакомиться по ссылке https://www.instagram.com/gifts.with.lovee/?hl=ru", reply_markup=keyboard) 
    
def create_keyboard_gifts():
    keyboard=types.InlineKeyboardMarkup(row_width=1)
    buttons=[types.InlineKeyboardButton(text=c[1], callback_data=c[0]) 
             for c in gifts.items()]
    keyboard.add(*buttons)
    return keyboard

def create_keyboard_yes():
    keyboard=types.InlineKeyboardMarkup(row_width=1)
    buttons=[types.InlineKeyboardButton(text=c[1], callback_data=c[0]) 
             for c in yes_no.items()]
    keyboard.add(*buttons)
    return keyboard

def create_keyboard_sizes():
    keyboard=types.InlineKeyboardMarkup(row_width=4)
    buttons=[types.InlineKeyboardButton(text=c[1], callback_data=c[0]) 
             for c in sizes.items()]
    keyboard.add(*buttons)
    return keyboard

def create_keyboard_cities():
    keyboard=types.InlineKeyboardMarkup(row_width=5)
    buttons=[types.InlineKeyboardButton(text=c[1], callback_data=c[0]) 
             for c in cities.items()]
    keyboard.add(*buttons)
    return keyboard

'''if (start==1):
    keyboard=create_keyboard_gifts()
    bot.send_message(message.chat.id, text="Выберите подарок С нашей коллекцией Вы можете ознакомиться по ссылке \nhttps://www.instagram.com/gifts.with.lovee/?hl=ru", reply_markup=keyboard) 
    start=0'''

@bot.message_handler(func = lambda message: get_state(message)==START)
def handle_start(message):
    keyboard=create_keyboard_gifts()
    bot.send_message(message.chat.id, text="Выберите новый подарок. \nС нашей коллекцией Вы можете ознакомиться по ссылке \nhttps://www.instagram.com/gifts.with.lovee/?hl=ru", reply_markup=keyboard) 
    
@bot.callback_query_handler(func = lambda callback_query: get_state(callback_query.message)==START)
def callhandler_start(callback_query):
    output=callback_query.data
    output=int(output)
    update_product(callback_query.message.chat.id, 'Title', gifts[output])
    update_state(callback_query.message, GIFT)
    keyboard=create_keyboard_sizes()
    bot.send_message(callback_query.message.chat.id, text="Выберите размер холста:", reply_markup=keyboard)
    #bot.send_message(chat_id=callback_query.message.chat.id, text="Подарок выбран")
    


@bot.callback_query_handler(func = lambda callback_query: get_state(callback_query.message)==GIFT)
def callhandler_start(callback_query):
    output=callback_query.data
    output=int(output)
    update_product(callback_query.message.chat.id, 'Size', sizes[output])
    update_state(callback_query.message, NAME)  
    bot.send_message(callback_query.message.chat.id, text="Как Вас зовут? Укажите свое имя.")


@bot.message_handler(func = lambda message: get_state(message)==NAME)
def handle_name(message):
    update_product(message.chat.id, 'Name', message.text)
    bot.send_message(message.chat.id, text="Укажите свой телефон.")
    update_state(message, PHONE)
    
    
@bot.message_handler(func = lambda message: get_state(message)==PHONE)
def handle_phone(message):
    update_product(message.chat.id, 'Phone', message.text)
    if ((message.text[0]!='+' and message.text[0]!='8'and message.text[0]!='7' and message.text[0]!='9') or len(message.text)<10):
        update_state(message, PHONE)
        bot.send_message(message.chat.id, text="Телефон должен начинаться с +7, 8, 9, и содержать не менее 10 символов. \nВведите телефон еще раз.")
    else:
        keyboard=create_keyboard_cities()
        bot.send_message(message.chat.id, text="Укажите свой город.", reply_markup=keyboard)
    
    
@bot.callback_query_handler(func = lambda callback_query: get_state(callback_query.message)==PHONE)
def callhandler_start(callback_query):
    output=callback_query.data
    output=int(output)
    update_product(callback_query.message.chat.id, 'City', cities[output])
    update_state(callback_query.message, FINISH) 
    product=get_product(callback_query.message.chat.id)
    keyboard=create_keyboard_yes()
    bot.send_message(callback_query.message.chat.id, text="Проверьте введенную информацию: \n- Подарок: {} \n- Размер холста: {} \n- Имя: {} \n- Телефон: {} \n- Город: {}".format(product["Title"], product["Size"],product["Name"], product["Phone"], product["City"]), reply_markup=keyboard)
    update_state(callback_query.message, FINISH)
    
    

@bot.callback_query_handler(func = lambda callback_query: get_state(callback_query.message)==FINISH)
def callhandler_start(callback_query):
    output=callback_query.data
    output=int(output)
    product=get_product(callback_query.message.chat.id)
    if yes_no[output]=='Отправить заявку':
        bot.send_message(callback_query.message.chat.id, text="Заявка отправлена. \nС вами свяжутся Alena&Yan")
        bot.send_message('270129913', text="Новый клиент: \n- Подарок: {} \n- Размер холста: {} \n- Имя: {} \n- Телефон: {} \n- Город: {}".format(product["Title"], product["Size"],product["Name"], product["Phone"], product["City"]))
        keyboard=create_keyboard_gifts()
        bot.send_message(callback_query.message.chat.id, text="***********************************")
        bot.send_message(callback_query.message.chat.id, text="Оставайтесь с нами.\nВыберите новый подарок. \nС нашей коллекцией Вы можете ознакомиться по ссылке \nhttps://www.instagram.com/gifts.with.lovee/?hl=ru", reply_markup=keyboard) 
    else:
        keyboard=create_keyboard_gifts()
        bot.send_message(callback_query.message.chat.id, text="Исправьте введенную информацию.\nВыберите подарок:", reply_markup=keyboard)
    update_state(callback_query.message, START)

    
    
bot.polling()


2019-11-20 02:18:11,488 (util.py:65 PollingThread) ERROR - TeleBot: "ReadTimeout occurred, args=(ReadTimeoutError("HTTPSConnectionPool(host='api.telegram.org', port=443): Read timed out. (read timeout=30)"),)
Traceback (most recent call last):
  File "/Users/torwayfarer/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py", line 384, in _make_request
    six.raise_from(e, None)
  File "<string>", line 2, in raise_from
  File "/Users/torwayfarer/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py", line 380, in _make_request
    httplib_response = conn.getresponse()
  File "/Users/torwayfarer/anaconda3/lib/python3.7/http/client.py", line 1321, in getresponse
    response.begin()
  File "/Users/torwayfarer/anaconda3/lib/python3.7/http/client.py", line 296, in begin
    version, status, reason = self._read_status()
  File "/Users/torwayfarer/anaconda3/lib/python3.7/http/client.py", line 257, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1"

ReadTimeout: HTTPSConnectionPool(host='api.telegram.org', port=443): Read timed out. (read timeout=30)

In [4]:
opening=['(','[','{']
closing=[')',']','}']

def parenthesis_checker(input):
    checker=[]
    for ch in input:
        if ch in opening:
            checker.append(ch)
        else:
            if (len(checker)>0 and checker[-1]==opening[closing.index(ch)]):
                checker.pop()
            else:
                return False        
    if len(checker)==0:
        return True
print(parenthesis_checker('()'))

True
